<a href="https://colab.research.google.com/github/orel33/bloc3/blob/master/reseaux/colab/traceroute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traceroute

Comment trouver son chemin sur le réseau Internet ?

## Prérequis

In [1]:
! apt-get -q -y install net-tools        # ifconfig 
! apt-get -q -y install dnsutils         # nslookup
! apt-get -q -y install traceroute       # traceroute
! apt-get -q -y install python3-scapy    # scapy

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 194 kB of archives.
After this operation, 803 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 net-tools amd64 1.60+git20161116.90da8a0-1ubuntu1 [194 kB]
Fetched 194 kB in 1s (199 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 132684 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20161116.90da8a0-1ubuntu1_amd64.deb ...
Unpacking net-tools (1.60+git20161116.90da8a0-1ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up net-tools (1.60+git20161116.90da8a0-1ubuntu1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  bind9-host geoip-database libb

## Qui suis-je ?

In [2]:
! ifconfig

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.28.0.2  netmask 255.255.0.0  broadcast 172.28.255.255
        ether 02:42:ac:1c:00:02  txqueuelen 0  (Ethernet)
        RX packets 2126  bytes 5820186 (5.8 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 1977  bytes 330235 (330.2 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 482  bytes 138426 (138.4 KB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 482  bytes 138426 (138.4 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



## Où vais-je ?

Sur le serveur web : http://www.perdu.com


In [3]:
! nslookup www.perdu.com

Server:		127.0.0.11
Address:	127.0.0.11#53

Non-authoritative answer:
Name:	www.perdu.com
Address: 208.97.177.124



## Traceroute (commande Unix)

In [4]:
! traceroute -I -4 -n www.perdu.com

traceroute to www.perdu.com (208.97.177.124), 30 hops max, 60 byte packets
 1  172.28.0.1  0.032 ms  0.009 ms  0.007 ms
 2  216.239.54.33  7.928 ms  8.007 ms  8.007 ms
 3  209.85.252.4  13.654 ms  13.748 ms  13.809 ms
 4  172.253.65.164  81.094 ms  81.155 ms  81.155 ms
 5  216.239.56.16  82.273 ms  82.272 ms  82.274 ms
 6  108.170.248.4  81.681 ms  81.648 ms  81.775 ms
 7  205.197.230.37  81.774 ms  80.580 ms  80.500 ms
 8  207.88.13.113  93.453 ms  88.222 ms  88.183 ms
 9  209.48.43.58  88.629 ms  88.493 ms  88.470 ms
10  208.113.156.4  90.186 ms  88.972 ms  89.068 ms
11  208.113.156.14  89.908 ms  89.819 ms  89.464 ms
12  208.97.177.124  88.376 ms  88.374 ms  88.437 ms


## Comment ça marche ?

Chaque paquet IP a une durée de vie initiale maximum, le champs TTL (Time To Leave), qui est décrémenté au passage de chaque routeur. Lorsque le TTL atteint 0, le routeur jette le paquet à la poubelle et émet un paquet ICMP d'erreur *Time to live exceeded* vers la source. 

* A quoi sert le TTL ?
* En déduire ainsi un moyen de découvrir les routeurs intermédiaires de proche en proche en incrémentant la valeur du TTL à partir de 1.

=> https://fr.wikipedia.org/wiki/Traceroute

=> https://en.wikipedia.org/wiki/Internet_Control_Message_Protocol


## Version Python/Scapy

=> https://scapy.readthedocs.io/en/latest/usage.html

Privilège *root* requis.

In [0]:
from scapy.all import *

def mytraceroute(target, maxttl):
    for k in range(1, maxttl):
        x = IP(version=4, dst=target, ttl=k)/ICMP()
        y = sr1(x, verbose=0)
        if y.getlayer(ICMP).type == 11 and y.getlayer(ICMP).code == 0:    # time to leave exceeded (11)
            print(k, y.src)
        elif y.getlayer(ICMP).type == 0 and y.getlayer(ICMP).code == 0:   # echo-reply (0)
            print(k, y.src)
            break
        else:
            print(k, "* * *")


mytraceroute("www.perdu.com", 64)

1 172.28.0.1
2 216.239.54.115
3 209.85.252.121
4 172.253.65.166
5 209.85.254.238
6 108.170.248.4
7 205.197.230.37
8 207.88.13.113
9 209.48.43.58
10 208.113.156.4
11 208.113.156.14
12 208.97.177.124
